# DATA ANNONATIONS

In [ ]:
import pandas as pd
import os
import cv2
from tqdm import tqdm

# Load CSV file
csv_file = "Licplatesdetection_train.csv"
df = pd.read_csv(csv_file)

# Define paths
image_folder = "Licplatesdetection_train\license_plates_detection_train"  # Folder containing car images
output_label_folder = "yolo_labels"  # Folder to save YOLO labels

# Create output folder if not exists
os.makedirs(output_label_folder, exist_ok=True)

# Process each image
for _, row in tqdm(df.iterrows(), total=len(df)):
    img_filename = str(row["img_id"])  # Example: "1.jpg", "2.jpg", etc.
    img_path = os.path.join(image_folder, img_filename)  # Full image path

    # Read image to get dimensions
    image = cv2.imread(img_path)
    if image is None:
        print(f"Warning: Image {img_filename} not found!")
        continue  # Skip missing images

    h, w, _ = image.shape  # Image height and width

    # Read bounding box coordinates
    xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]

    # Convert to YOLO format (normalized values between 0 and 1)
    x_center = (xmin + xmax) / (2 * w)
    y_center = (ymin + ymax) / (2 * h)
    width = (xmax - xmin) / w
    height = (ymax - ymin) / h

    # Save to YOLO label file (same name as image but with .txt)
    label_filename = img_filename.replace(".jpg", ".txt")
    label_path = os.path.join(output_label_folder, label_filename)

    with open(label_path, "w") as f:
        f.write(f"0 {x_center} {y_center} {width} {height}\n")  # "0" is class_id for license plate

print("✅ Conversion completed! YOLO labels are saved in:", output_label_folder)


# Train -Val Split

In [ ]:
import os
import shutil
import random
import pandas as pd

# Define paths
image_folder = "Licplatesdetection_train\license_plates_detection_train"  # Folder containing car images
label_folder = "yolo_labels"  # Folder containing YOLO labels

output_dir = "dataset"
train_img_folder = os.path.join(output_dir, "images/train")
val_img_folder = os.path.join(output_dir, "images/val")
train_label_folder = os.path.join(output_dir, "labels/train")
val_label_folder = os.path.join(output_dir, "labels/val")

# Create output directories if not exist
for folder in [train_img_folder, val_img_folder, train_label_folder, val_label_folder]:
    os.makedirs(folder, exist_ok=True)

# Get all images and shuffle
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]
random.shuffle(image_files)

# Define split ratio
split_index = int(0.8 * len(image_files))  # 80% for training

train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Function to move images and labels
def move_files(file_list, dest_img_folder, dest_label_folder):
    for img_file in file_list:
        img_path = os.path.join(image_folder, img_file)
        label_path = os.path.join(label_folder, img_file.replace(".jpg", ".txt"))

        # Move images
        if os.path.exists(img_path):
            shutil.move(img_path, os.path.join(dest_img_folder, img_file))

        # Move corresponding labels
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_label_folder, img_file.replace(".jpg", ".txt")))

# Move files
move_files(train_files, train_img_folder, train_label_folder)
move_files(val_files, val_img_folder, val_label_folder)

print("✅ Dataset split completed!")
print(f"Train set: {len(train_files)} images")
print(f"Validation set: {len(val_files)} images")


In [ ]:
/dataset
    /images
        /train  (contains 720 training images)
        /val  (contains 180 validation images)
    /labels
        /train  (contains 720 YOLO annotation files)
        /val  (contains 180 YOLO annotation files)


In [ ]:
"data.yaml"
train: /path/to/dataset/images/train
val: /path/to/dataset/images/val
nc: 1
names: ["license_plate"]


In [ ]:
# Install YOLOv5 and dependencies
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


In [ ]:
/your_dataset_folder
  /images
    /train
    /val
  /labels
    /train
    /val
  data.yaml


In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data data.yaml --weights yolov5s.pt


In [ ]:
!cp runs/train/exp/weights/lapi.pt /content/drive/MyDrive/yolov5_trained.pt
